In [113]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences

In [121]:
def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def print_obj(obj, i=1, params=["agent", "interaction", "space", "time", "causal link", "physical law"]):
    print(f"{i} - ")
    for param in params:
        print(f"{param}: {obj[param]}")
    

def parse_func_resp(resp, obj_name="element", params=["name", "text"], print="True"):
    args_raw = resp.choices[0].message.tool_calls[0].function.arguments
    raws = json.loads(args_raw)[obj_name]

    obj_lst = []
    for i, obj in enumerate(raws, 1):
        if print:
            pass
            #print_obj(obj, i, params)
        obj_lst.append(obj)
    
    return obj_lst

def elements_to_string(dims, label="ELEMENTS"):
    s = ""
    for dim, elems in dims.items():
        s += f"{dim}\n{label}:\n"
        for elem in elems:
            s += elem + "\n"
        s += "\n"

    return s

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt

def ask_func(sys_msg, usr_msg, FUNC):
    resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
            tools=[FUNC], tool_choice="auto", temperature=0.5
            )
    return resp

def sample_elems(elements, per_dim, per_elem):
    total_dim = len(elements)
    ndims = min(int(per_dim * total_dim), total_dim)

    # Sample dimension indices
    dimension_indices = random.sample(range(total_dim), ndims)

    sample_elems = {}

    for di in dimension_indices:
        dimension_name = list(elements.keys())[di]
        dimension_elems = list(elements.values())[di]

        total_elems = len(dimension_elems)
        nelems = min(int(per_elem * total_elems), total_elems)

        # Sample element indices from this dimension
        element_indices = random.sample(range(total_elems), nelems)
        sample_elems[dimension_name] = [dimension_elems[ei] for ei in element_indices]

    return sample_elems

def clean_elements(elements_raw):
    elements = {}
    for element_raw in elements_raw:
        k = element_raw["name"]
        v = element_raw["text"]
        element = (k, [item.strip() for item in v.split('\n') if item.strip()])
        elements[element[0]] = element[1]
    return elements
    



load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library.sqlite"
MAX_RESULTS = 100

In [115]:
class story(BaseModel):
    agent: str
    interaction: str
    space: str
    time: str
    link: str
    law: str

    
SYS_PROMPT_ELEMENTS = """
My goal is a description of the mechanical forces driving the development of the hindgut and an understanding of how the overexpression of myosin 1c drives the inversion of LR asymmetry 
during development. I want to write a complete story of hindgut development incorporating mechanical force as central players in this story, 
then create a complete story of hindgut development under Myo1C OE, the generation of LR inversion from the role of Myo1C to the hindgut rotation. 
The following text is from a paper on hindgut morphogenesis.
Extract a list of agents, interactions, spatial compartments, developmental stages, causal chains, and physical constraints involved in hindgut morphogenesis and the effect of myosin 1c overexpression on LR inversion.
Structure the output as a story scaffold:
Agents: ...
Interactions: ...
Spaces: ...
Time: ...
Causal links: ...
Physical laws/constraints: ...
"""


SYS_PROMPT_DIMENSIONS = """
You are helping build a complete narrative of hindgut morphogenesis in Drosophila, with special attention to the role of mechanical forces - driving the rotation and elongation of the hindgut - and the overexpression of myosin 1c (Myo1C) in left-right (LR) inversion.

Your task is not to tell the story, but to define what would be required to tell a **complete and comprehensive story** for this set of story elements.

- Break down these elements into their essential **story dimensions** or **sub-components**.
- Think about biological scale (molecular, cellular, tissue), space, time, dynamics, agents, interactions, functions, and unknowns.
- Consider how these elements might relate to:
  - other parts of the hindgut
  - the mechanical forces shaping the tissue
  - the overexpression of Myo1C and its effects on LR inversion

Output a structured list of **story dimensions** that should be explored in order to fully develop these elements. If helpful, include example questions or variables within each dimension.

Format:

**1. Dimension Name**  
- Description of what this dimension involves.  
- Example questions or aspects to explore.

(Repeat as needed for all relevant dimensions)
"""


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")


SYS_PROMPT_STORYTELLING = """
You are constructing a comprehensive and mechanistically rich narrative of **Drosophila hindgut morphogenesis** from stage 11 to stage 15, with a focus on the role of mechanical forces driving the left-right asymmetric rotation and anterior-posterior elongation of the gut.

The goal is to create a structured, multi-dimensional story that:
- Spans molecular, cellular, tissue, and system levels.
- Integrates all known biological elements (agents, interactions, causal links, physical laws, etc.).
- Is told along a set of distinct conceptual **dimensions** such as spatial organization, genetic regulation, tissue morphogenesis, and unknowns.

---

### 🔬 BIOLOGICAL ELEMENTS TO INTEGRATE
The following are biological elements grouped by type. These must be incorporated into the story in a mechanistic, causally connected way. Explain how they participate in hindgut development and/or LR inversion, referencing the correct level of biological organization and linking them logically across space and time.

{elements_text}

---

### 🧭 DIMENSIONS OF STORY STRUCTURE
Organize the story along the following dimensions. For each dimension, describe how it helps us understand the listed elements and their roles in morphogenesis and asymmetry inversion. Address guiding questions where provided.

{dimensions_text}

---

### 📘 INSTRUCTIONS FOR STORYTELLING

- Tell a unified story that moves through all the listed dimensions while referencing the relevant elements.
- Ensure causal consistency, developmental sequencing, and physical plausibility.
- Emphasize the interplay between mechanical forces, gene expression, protein interactions, cellular behaviors, and tissue-level morphogenesis.
- Highlight where Myo1C overexpression introduces asymmetry inversion or disrupts normal patterns.
- Use scientific language and structure the story in clear sections by dimension.

End the narrative with:
- A synthesis of how these dimensions collectively explain hindgut morphogenesis.
- A brief discussion of unresolved questions and possible directions for experimental investigation.
"""

SYS_PROMPT_STORYTELLING2 = """
You are rewriting the following comprehensive story of Drosophila hindgut morphogenesis from stage 11 to stage 15, which incorporates the molecular, cellular, and tissue-level dynamics that drive normal left-right (LR) asymmetric development.

---

**Original Story:**
{full_story}

---

### 🔄 Mutation to Apply:
Rewrite the story under the condition that **Myosin 1C (Myo1C) is overexpressed across the brachyenteron domain leading to the inversion of LR asymmetries**.

### 🎯 Objective:
- Show how this overexpression perturbs or rewires the original causal and developmental dynamics.
- Identify **which molecular, cellular, and tissue-level elements are affected**, and how their behavior, interactions, or timing change.
- Describe **how these changes lead to inversion of LR asymmetry** in the hindgut.
- Adjust the **temporal dynamics, mechanical forces, gene expression outcomes, and spatial organization** as needed to reflect the overexpression.
- Maintain the structure and dimensional sections of the original story (e.g., Molecular Interactions, Cellular Dynamics, etc.), but adapt their content to reflect the mutant condition.

### 📌 Guidelines:
- Be specific and mechanistic—do not merely say “Myo1C disrupts development.”
- Describe how Myo1C activity plays a role in each element, each dimension laid out in the original story.
- Clearly identify **causal changes**, new feedbacks, or failure points introduced by Myo1C OE in the brachyenteron domain.
- Where evidence is lacking, hypothesize plausible explanations.
- Conclude with a revised synthesis and note the experimental implications of this inversion scenario.

You are writing a new, mutated version of the story—show how a single molecular perturbation cascades into a full organ-level inversion of developmental outcome.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}

In [10]:
KEYWORDS = ["active tension network", "epithelial"]  # list of words
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# papers = {}
for r in search_results:
    title = r['title']
    text = r['text']
    print(title, len(text))

Active Tension Network Model in Epithelial Tissues 32334
The Geometric Basis of Epithelial Convergent Extension 188479
Generating Active T1 Transitions through Mechanochemical Feedback 117183


In [9]:
KEYWORDS = ["active tension network", "epithelial"]  # list of words
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# papers = {}
for r in search_results:
    title = r['title']
    text = r['text']
    papers[title] = text


rem = [None, "Fundamental Neuroscience (3rd edition)","Harrison's Principles of Internal Medicine 15th Edition", "Developmental Biology, 7th Edition"]
for r in rem:
    if r in papers:
        del papers[r]


for title in list(papers.keys()):
    print(title)

The Drosophila actin nucleator DAAM is essential for left-right asymmetry
Myosin1D is an evolutionarily conserved regulator of animal left–right asymmetry
An unconventional myosin in Drosophila reverses the default handedness in visceral organs
Chiral cell sliding drives left-right asymmetric organ twisting
Cell Chirality Drives Left-Right Asymmetric Morphogenesis
Chirality in Planar Cell Shape Contributes to Left-Right Asymmetric Epithelial Morphogenesis
Vertebrate myosin 1d regulates left–right organizer morphogenesis and laterality
Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila
Understanding Laterality Disorders and the Left-Right Organizer: Insights from Zebrafish
Generating Active T1 Transitions through Mechanochemical Feedback
Left–right Myosin-Is, Myosin1C, and Myosin1D exhibit distinct single molecule behaviors on the plasma membrane of Drosophila macrophages
It Takes Guts: The Drosophila Hindgut a

In [123]:
wt_stories = {}
myo_stories = {}

for n in range(len(papers)):
    title, text = list(papers.keys())[n], list(papers.values())[n]
    excerpt = sample_text(text, 10000)

    sys_msg = SYS_PROMPT_ELEMENTS
    user_msg = f"TEXT EXCERPT:\n{excerpt}"
    elements_resp = ask(sys_msg, user_msg)

    sys_msg = SYS_PROMPT_PARSE_ELEMENTS
    usr_msg = f"STORY FRAMEWORK:\n{elements_resp}"
    elements_func = ask_func(sys_msg, usr_msg, ELEM_FUNC)

    elements_raw = parse_func_resp(elements_func, "element", ["name", "text"], "False")
    elements = clean_elements(elements_raw)

    per_dim, per_elem = 1, 1   # Number of elements per dimension to sample
    elements_sample = sample_elems(elements, per_dim, per_elem)
    elements_text = elements_to_string(elements_sample, "ELEMENTS")

    # GET DIMENSIONS
    sys_msg = SYS_PROMPT_DIMENSIONS
    usr_msg = f"Here are the story elements:\n\n{elements_text}"
    dimensions_resp = ask(sys_msg, usr_msg)

    sys_msg = SYS_PROMPT_PARSE_ELEMENTS
    usr_msg = f"STORY DIMENSIONS:\n{dimensions_resp}"
    dimensions_func = ask_func(sys_msg, usr_msg, ELEM_FUNC)

    dimensions_raw = parse_func_resp(dimensions_func, "element", ["name", "text"], "False")
    dimensions = clean_elements(dimensions_raw)

    # Configuration
    per_dim, per_elem = 1, 1 
    dimensions_sample = sample_elems(dimensions, per_dim, per_elem)
    dimensions_text = elements_to_string(dimensions_sample, "DIMENSIONS")

    sys_msg = SYS_PROMPT_STORYTELLING.format(elements_text=elements_text, dimensions_text=dimensions_text)
    usr_msg = "Please generate the complete, structured narrative based on the elements and dimensions provided."
    wt_resp = ask(sys_msg, usr_msg)

    sys_msg = SYS_PROMPT_STORYTELLING2.format(full_story=wt_resp)
    usr_msg = "Please rewrite the original story to reflect the overexpression of Myosin 1C (Myo1C) across the brachyenteron domain. Show how this perturbation leads to inversion of left-right asymmetry in the Drosophila hindgut by altering the behavior of affected molecular, cellular, and tissue-level elements. Maintain the original story structure and focus on causal, mechanistic explanations."
    myo_resp = ask(sys_msg, usr_msg)

    wt_stories[title] = {
        "elements": elements_sample,
        "dimensions": dimensions_sample,
        "story": wt_resp
    }
    myo_stories[title] = {
        "mutated_story": myo_resp
    }


    print(title, "\n\n")
    print(wt_resp, "\n\n")
    print(myo_resp, "\n\n\n")
    


    



    

    


    





KeyboardInterrupt: 

In [119]:
print(sample_elems)

<function sample_elems at 0x12091e160>


In [24]:
sys_msg = SYS_PROMPT_PARSE_ELEMENTS

usr_msg = f"STORY FRAMEWORK:\n{resp}"

resp2 = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
        tools=[ELEM_FUNC], tool_choice="auto", temperature=0.5
)

In [29]:
elements_raw = parse_func_resp(resp2, "element", ["name", "text"])
elements = {k: [item.strip() for item in v.split('\n')] for k, v in event} # convert raw into a list

1 - 
name: Agents
text: - DAAM (Disheveled-Associated Activator of Morphogenesis)  
- Myo1D (Myosin 1D)  
- DE-cadherin  
- β-catenin  
- Gal4/UAS system  
- Abd-B (Abdominal-B Hox gene)  
- Profilin  
- Tec29  
- Actin Nucleators (e.g., other formins, Arp2/3 complex)
2 - 
name: Interactions
text: - DAAM interacts with Myo1D as part of a protein complex in left-right (LR) organizers.  
- Myo1D binds to DE-cadherin and β-catenin, suggesting docking at adherens junctions.  
- DAAM's involvement in actin nucleation interacts with Profilin and G-actin to nucleate F-actin.  
- DAAM and Myo1D genetically interact, specifically in the context of LR asymmetry and genitalia rotation.  
- The regulation of DAAM by Hox gene Abd-B suggests additional genetic interactions.
3 - 
name: Spaces
text: - LR organizers in Drosophila (e.g., imaginal rings, genital plate, hindgut)  
- Specific cell membranes and DE-cadherin-rich adherens junctions  
- Genitalia and hindgut during developmental stages
4 - 
n

In [32]:
print(elements_raw)

[{'name': 'Agents', 'text': '- DAAM (Disheveled-Associated Activator of Morphogenesis)  \n- Myo1D (Myosin 1D)  \n- DE-cadherin  \n- β-catenin  \n- Gal4/UAS system  \n- Abd-B (Abdominal-B Hox gene)  \n- Profilin  \n- Tec29  \n- Actin Nucleators (e.g., other formins, Arp2/3 complex)'}, {'name': 'Interactions', 'text': "- DAAM interacts with Myo1D as part of a protein complex in left-right (LR) organizers.  \n- Myo1D binds to DE-cadherin and β-catenin, suggesting docking at adherens junctions.  \n- DAAM's involvement in actin nucleation interacts with Profilin and G-actin to nucleate F-actin.  \n- DAAM and Myo1D genetically interact, specifically in the context of LR asymmetry and genitalia rotation.  \n- The regulation of DAAM by Hox gene Abd-B suggests additional genetic interactions."}, {'name': 'Spaces', 'text': '- LR organizers in Drosophila (e.g., imaginal rings, genital plate, hindgut)  \n- Specific cell membranes and DE-cadherin-rich adherens junctions  \n- Genitalia and hindgut d

In [36]:
elements = {}
for element_raw in elements_raw:
    k = element_raw["name"]
    v = element_raw["text"]
    element = (k, [item.strip() for item in v.split('\n') if item.strip()])
    elements[element[0]] = element[1]

    
print(elements)

{'Agents': ['- DAAM (Disheveled-Associated Activator of Morphogenesis)', '- Myo1D (Myosin 1D)', '- DE-cadherin', '- β-catenin', '- Gal4/UAS system', '- Abd-B (Abdominal-B Hox gene)', '- Profilin', '- Tec29', '- Actin Nucleators (e.g., other formins, Arp2/3 complex)'], 'Interactions': ['- DAAM interacts with Myo1D as part of a protein complex in left-right (LR) organizers.', '- Myo1D binds to DE-cadherin and β-catenin, suggesting docking at adherens junctions.', "- DAAM's involvement in actin nucleation interacts with Profilin and G-actin to nucleate F-actin.", '- DAAM and Myo1D genetically interact, specifically in the context of LR asymmetry and genitalia rotation.', '- The regulation of DAAM by Hox gene Abd-B suggests additional genetic interactions.'], 'Spaces': ['- LR organizers in Drosophila (e.g., imaginal rings, genital plate, hindgut)', '- Specific cell membranes and DE-cadherin-rich adherens junctions', '- Genitalia and hindgut during developmental stages'], 'Time': ['- Pupal 

In [101]:
# use n % elements, to get a list of dimensions of the story of the story of hindgut morphogenesis

# Configuration
per_dim = 1     # Number of dimensions to sample (as a fraction or fixed count)
per_elem = 1    # Number of elements per dimension to sample

# Get total dimensions
total_dim = len(elements)
ndims = min(int(per_dim * total_dim), total_dim)

# Sample dimension indices
dimension_indices = random.sample(range(total_dim), ndims)

sample_elems = {}

for di in dimension_indices:
    dimension_name = list(elements.keys())[di]
    dimension_elems = list(elements.values())[di]

    total_elems = len(dimension_elems)
    nelems = min(int(per_elem * total_elems), total_elems)

    # Sample element indices from this dimension
    element_indices = random.sample(range(total_elems), nelems)
    sample_elems[dimension_name] = [dimension_elems[ei] for ei in element_indices]


str_elems = elements_to_string(sample_elems)
print(str_elems)


# get dimensions








Time
ELEMENTS:
- A critical window between days 5-6 of development for the requirement of Myo1D and DAAM in genitalia LR organizers
- Pupal stage during Drosophila development

Physical laws/constraints
ELEMENTS:
- Constraints of actin dynamics, nucleation, and organization drive morphological changes during symmetry breaking.
- Adherens junction functionality as sites for potential transmission of mechanical forces involved in the asymmetric development process.
- The protein complex formation and membrane localization underline the spatial requirements for DAAM and Myo1D activity.

Interactions
ELEMENTS:
- DAAM and Myo1D genetically interact, specifically in the context of LR asymmetry and genitalia rotation.
- DAAM's involvement in actin nucleation interacts with Profilin and G-actin to nucleate F-actin.
- DAAM interacts with Myo1D as part of a protein complex in left-right (LR) organizers.
- Myo1D binds to DE-cadherin and β-catenin, suggesting docking at adherens junctions.
- The r

In [79]:
sys_msg = SYS_PROMPT_DIMENSIONS.format(elems_string=elems_string)
resp3 = ask(sys_msg, elems_string)
print(resp3)

**1. Molecular Interactions**  
- Description: Explore the roles and interactions of molecular agents involved in hindgut morphogenesis and LR asymmetry.  
- Example questions or aspects to explore:
  - How does Myo1D influence actin rearrangement at a molecular level?
  - What are the molecular pathways through which β-catenin and Tec29 impact hindgut development?
  - How does the Gal4/UAS system allow precise control of gene expression in this context?

**2. Cellular Dynamics**  
- Description: Investigate the cellular changes and behaviors during the process of hindgut morphogenesis.  
- Example questions or aspects to explore:
  - What cell types and behaviors (e.g., proliferation, migration, differentiation) are involved in hindgut morphogenesis?
  - How do mechanical forces at the cellular level contribute to hindgut rotation and elongation?

**3. Tissue-Level Morphogenesis**  
- Description: Examine how cellular processes integrate to drive changes at the tissue level, resulting

In [80]:
sys_msg = SYS_PROMPT_PARSE_ELEMENTS

usr_msg = f"STORY FRAMEWORK:\n{resp3}"

resp4 = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
        tools=[ELEM_FUNC], tool_choice="auto", temperature=0.5
)

In [112]:
sys_msg = SYS_PROMPT_PARSE_ELEMENTS

usr_msg = f"STORY FRAMEWORK:\n{resp3}"

resp4 = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
        tools=[ELEM_FUNC], tool_choice="auto", temperature=0.5
)

dimensions_raw = parse_func_resp(resp4, "element", ["name", "text"])
#elements = {k: [item.strip() for item in v.split('\n')] for k, v in event} # convert raw into a list

1 - 
name: 1. Molecular Interactions
text: - Description: Explore the roles and interactions of molecular agents involved in hindgut morphogenesis and LR asymmetry.  
- Example questions or aspects to explore:
  - How does Myo1D influence actin rearrangement at a molecular level?
  - What are the molecular pathways through which β-catenin and Tec29 impact hindgut development?
  - How does the Gal4/UAS system allow precise control of gene expression in this context?
2 - 
name: 2. Cellular Dynamics
text: - Description: Investigate the cellular changes and behaviors during the process of hindgut morphogenesis.  
- Example questions or aspects to explore:
  - What cell types and behaviors (e.g., proliferation, migration, differentiation) are involved in hindgut morphogenesis?
  - How do mechanical forces at the cellular level contribute to hindgut rotation and elongation?
3 - 
name: 3. Tissue-Level Morphogenesis
text: - Description: Examine how cellular processes integrate to drive changes

In [85]:
print(dimensions_raw)

[{'name': '1. Molecular Interactions', 'text': '- Description: Explore the roles and interactions of molecular agents involved in hindgut morphogenesis and LR asymmetry.  \n- Example questions or aspects to explore:\n  - How does Myo1D influence actin rearrangement at a molecular level?\n  - What are the molecular pathways through which β-catenin and Tec29 impact hindgut development?\n  - How does the Gal4/UAS system allow precise control of gene expression in this context?'}, {'name': '2. Cellular Dynamics', 'text': '- Description: Investigate the cellular changes and behaviors during the process of hindgut morphogenesis.  \n- Example questions or aspects to explore:\n  - What cell types and behaviors (e.g., proliferation, migration, differentiation) are involved in hindgut morphogenesis?\n  - How do mechanical forces at the cellular level contribute to hindgut rotation and elongation?'}, {'name': '3. Tissue-Level Morphogenesis', 'text': '- Description: Examine how cellular processes 

In [83]:
print(elements_raw)

[{'name': '1. Molecular Interactions', 'text': '- Description: Explore the roles and interactions of molecular agents involved in hindgut morphogenesis and LR asymmetry.  \n- Example questions or aspects to explore:\n  - How does Myo1D influence actin rearrangement at a molecular level?\n  - What are the molecular pathways through which β-catenin and Tec29 impact hindgut development?\n  - How does the Gal4/UAS system allow precise control of gene expression in this context?'}, {'name': '2. Cellular Dynamics', 'text': '- Description: Investigate the cellular changes and behaviors during the process of hindgut morphogenesis.  \n- Example questions or aspects to explore:\n  - What cell types and behaviors (e.g., proliferation, migration, differentiation) are involved in hindgut morphogenesis?\n  - How do mechanical forces at the cellular level contribute to hindgut rotation and elongation?'}, {'name': '3. Tissue-Level Morphogenesis', 'text': '- Description: Examine how cellular processes 

In [86]:
sys_msg = SYS_PROMPT_PARSE_ELEMENTS

usr_msg = f"STORY FRAMEWORK:\n{resp3}"

resp4 = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
        tools=[ELEM_FUNC], tool_choice="auto", temperature=0.5
)

dimensions_raw = parse_func_resp(resp4, "element", ["name", "text"])

dimensions = {}
for dimension_raw in dimensions_raw:
    k = dimension_raw["name"]
    v = dimension_raw["text"]
    dimension = (k, [item.strip() for item in v.split('\n') if item.strip()])
    dimensions[dimension[0]] = dimension[1]

print(dimensions)

{'1. Molecular Interactions': ['- Description: Explore the roles and interactions of molecular agents involved in hindgut morphogenesis and LR asymmetry.', '- Example questions or aspects to explore:', '- How does Myo1D influence actin rearrangement at a molecular level?', '- What are the molecular pathways through which β-catenin and Tec29 impact hindgut development?', '- How does the Gal4/UAS system allow precise control of gene expression in this context?'], '2. Cellular Dynamics': ['- Description: Investigate the cellular changes and behaviors during the process of hindgut morphogenesis.', '- Example questions or aspects to explore:', '- What cell types and behaviors (e.g., proliferation, migration, differentiation) are involved in hindgut morphogenesis?', '- How do mechanical forces at the cellular level contribute to hindgut rotation and elongation?'], '3. Tissue-Level Morphogenesis': ['- Description: Examine how cellular processes integrate to drive changes at the tissue level, 

In [96]:
# use n % dimension, to write the story of hindgut morphogenesis

import random

# Configuration
per_dim = 1     # Number of dimensions to sample (as a fraction or fixed count)
per_elem = 1    # Number of elements per dimension to sample

# Get total dimensions
total_dim = len(dimensions)
ndims = min(int(per_dim * total_dim), total_dim)

# Sample dimension indices
dimension_indices = random.sample(range(total_dim), ndims)

sample_dims = {}

for di in dimension_indices:
    dimension_name = list(dimensions.keys())[di]
    dimension_elems = list(dimensions.values())[di]

    total_elems = len(dimension_elems)
    nelems = min(int(per_elem * total_elems), total_elems)

    # Sample element indices from this dimension
    element_indices = random.sample(range(total_elems), nelems)
    sample_dims[dimension_name] = [dimension_elems[ei] for ei in element_indices]


str_dims = elements_to_string(sample_dims, "DIMENSIONS")
print(str_dims)

# get dimensions


2. Cellular Dynamics
DIMENSIONS:
- Example questions or aspects to explore:
- Description: Investigate the cellular changes and behaviors during the process of hindgut morphogenesis.
- How do mechanical forces at the cellular level contribute to hindgut rotation and elongation?
- What cell types and behaviors (e.g., proliferation, migration, differentiation) are involved in hindgut morphogenesis?

1. Molecular Interactions
DIMENSIONS:
- Description: Explore the roles and interactions of molecular agents involved in hindgut morphogenesis and LR asymmetry.
- What are the molecular pathways through which β-catenin and Tec29 impact hindgut development?
- Example questions or aspects to explore:
- How does the Gal4/UAS system allow precise control of gene expression in this context?
- How does Myo1D influence actin rearrangement at a molecular level?

6. Genetic Regulation and Overexpression Effects
DIMENSIONS:
- What genetic factors regulate Myo1C expression and its impact on hindgut devel

In [97]:
str_elems = elems_string
str_dims = dims_string

In [102]:
print(str_elems)
print(str_dims)

Time
ELEMENTS:
- A critical window between days 5-6 of development for the requirement of Myo1D and DAAM in genitalia LR organizers
- Pupal stage during Drosophila development

Physical laws/constraints
ELEMENTS:
- Constraints of actin dynamics, nucleation, and organization drive morphological changes during symmetry breaking.
- Adherens junction functionality as sites for potential transmission of mechanical forces involved in the asymmetric development process.
- The protein complex formation and membrane localization underline the spatial requirements for DAAM and Myo1D activity.

Interactions
ELEMENTS:
- DAAM and Myo1D genetically interact, specifically in the context of LR asymmetry and genitalia rotation.
- DAAM's involvement in actin nucleation interacts with Profilin and G-actin to nucleate F-actin.
- DAAM interacts with Myo1D as part of a protein complex in left-right (LR) organizers.
- Myo1D binds to DE-cadherin and β-catenin, suggesting docking at adherens junctions.
- The r

In [106]:
sys_msg = SYS_PROMPT_STORYTELLING.format(elements_text=str_elems, dimensions_text=str_dims)
usr_msg = "Please generate the complete, structured narrative based on the elements and dimensions provided."

resp_story = ask(sys_msg, usr_msg)
print(resp_story)



**Title: A Multi-Dimensional Tale of Drosophila Hindgut Morphogenesis and Left-Right Asymmetry**

### 1. Molecular Interactions

Hindgut morphogenesis in Drosophila is a marvel of molecular choreography, wherein **DAAM (Disheveled-Associated Activator of Morphogenesis)** and **Myo1D (Myosin 1D)** play pivotal roles. Their interactions, particularly the binding of Myo1D to **DE-cadherin** and **β-catenin**, localize these proteins at adherens junctions—crucial for transmitting mechanical forces. **DAAM**, an actin nucleator, operates alongside **Profilin** and **G-actin** to nucleate F-actin structures. These structures are vital for generating the mechanical forces necessary for the symmetry breaking that precedes the distinctive rotational morphogenesis of the hindgut. The protein complex formed by DAAM and Myo1D in left-right (LR) organizers retains spatial specificity, critical for their functions.

**Tec29**, another molecular agent, is hypothesized to interact with β-catenin and c

In [109]:
sys_msg = SYS_PROMPT_STORYTELLING2.format(full_story=resp_story)
usr_msg = "Please rewrite the original story to reflect the overexpression of Myosin 1C (Myo1C) across the brachyenteron domain. Show how this perturbation leads to inversion of left-right asymmetry in the Drosophila hindgut by altering the behavior of affected molecular, cellular, and tissue-level elements. Maintain the original story structure and focus on causal, mechanistic explanations."

resp_myo = ask(sys_msg, usr_msg)
print(resp_myo)


### Title: Disrupted Dynamics and Inversion of Asymmetry in Drosophila Hindgut Morphogenesis due to Myosin 1C Overexpression

### 1. Molecular Interactions

Hindgut morphogenesis in Drosophila normally relies on precise molecular interactions, but overexpression of **Myosin 1C (Myo1C)** across the brachyenteron domain profoundly perturbs this process. Typically, **Myo1D** interacts with **DE-cadherin** and **β-catenin** to localize mechanical forces at adherens junctions; however, the introduction of high levels of Myo1C disrupts these critical interactions. Myo1C competitively binds to adherens junction components, potentially displacing or mislocalizing essential proteins like DAAM at junctions, and rewires the profile of actin nucleation, altering the structure formation required for normal rotation and LR asymmetry.

The **DAAM**-induced actin dynamics now interact with Myo1C instead of Myo1D, creating aberrant F-actin structures. These structures are misaligned, disrupting the typ